In [ ]:
%cd ..
import math
import numpy as np
import time
import torch
from typing import Any
from archive.python.LangevinGillespie import LangevinGillespie
from src.utils.compute_transition_matrix import compute_transition_matrix
from f1sim import LangevinGillespie as LangevinGillespie_PybindWrap


/home/Robert/Code/Python


/usr/lib/python3/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


ModuleNotFoundError: No module named 'f1sim'

In [ ]:
!fastfetch --logo none --structure os:kernel:cpu:gpu

In [ ]:
def initialize_simulation_params(LG):
    LG.steps = 2000
    LG.dt = 1e-6
    LG.method = "heun"

    # Mechanical / Thermal Setup
    LG.kappa = 56
    LG.kBT = 4.14
    LG.gammaB = LG.computeGammaB(a=20, r=19, eta=1e-9)

    # Multi State Setup
    LG.theta_states = np.array([3, 36, 72, 116]) * math.pi / 180  # Deg → Rad
    LG.initial_state = 0  # Starting state

    # Transition rate matrix
    LG.transition_matrix = compute_transition_matrix(LG)


def compute_simulation_time(obj: Any, n_steps: int, rng_seed: Any = None) -> float:
    """Run a simulation with n_steps on `obj` and return elapsed time in seconds."""
    obj.steps = n_steps
    start_time = time.time()
    obj.simulate(rng_seed)
    return time.time() - start_time


def run_simulations(LG, MAX_STEPS):
    step_counts = list(range(1, MAX_STEPS + 1))
    times = np.array([])

    print("Running simulations...")
    for i, steps in enumerate(step_counts):
        elapsed = compute_simulation_time(LG, steps, rng_seed=42)
        times = np.append(times, elapsed)

        print(f"\rStep {i}/{len(step_counts)}", end="", flush=True)

    times_sum, times_mean = times.sum(), times.mean()
    print("\nSimulations complete!")
    print(f"Total Time to compute {times_sum} seconds")
    print(f"Average time to compute {times_mean} seconds")
    return times_sum, times_mean

In [ ]:
# Initialize simulation wrapper
LG_PybindWrap = LangevinGillespie_PybindWrap()
initialize_simulation_params(LG_PybindWrap)

N_SIMS = 100_000

BYTES_PER_FLOAT = 4
BYTES_PER_INT = 4
TOTAL_FLOATS_PER_SIM = 3 * LG_PybindWrap.steps  # bead_positions, target_theta, etc.
TOTAL_INTS_PER_SIM = LG_PybindWrap.steps  # states
BYTES_PER_SIM = (
    TOTAL_FLOATS_PER_SIM * BYTES_PER_FLOAT + TOTAL_INTS_PER_SIM * BYTES_PER_INT
)

# --- Dynamic GPU memory check ---
if torch.cuda.is_available():
    device = torch.device("cuda")
    total_mem = torch.cuda.get_device_properties(0).total_memory
    allocated = torch.cuda.memory_allocated(0)
    reserved = torch.cuda.memory_reserved(0)
    free_mem = total_mem - allocated - reserved

    # Leave some room for other GPU usage
    MAX_MEMORY_BYTES = int(free_mem * 0.5)
else:
    # fallback for CPU only
    MAX_MEMORY_BYTES = 8 * 1024**3  # GB

# Compute batch size
BATCH_SIZE = max(1, min(N_SIMS, MAX_MEMORY_BYTES // BYTES_PER_SIM))
total_batches = math.ceil(N_SIMS / BATCH_SIZE)

print(f"Batch size: {BATCH_SIZE}, total batches: {total_batches}")


##### Note: If you use multi-threading, avoid using swap memory, instead employ batching

In [ ]:

start_time = time.time()
for batch_start in range(0, N_SIMS, BATCH_SIZE):
    n_batch = min(BATCH_SIZE, N_SIMS - batch_start)
    batch_num = batch_start // BATCH_SIZE + 1

    print(f"\rRunning batch {batch_num}/{total_batches}: {n_batch} simulations", end="", flush=True)

    # Run CUDA kernel
    beads, states, thetas = LG_PybindWrap.simulate_multithreaded_cuda(n_batch)

print(f"\nTotal time: {time.time() - start_time:.2f} s")


In [ ]:
# start_time = time.time()
# for batch_start in range(0, N_SIMS, BATCH_SIZE):
#     n_batch = min(BATCH_SIZE, N_SIMS - batch_start)
#     batch_num = batch_start // BATCH_SIZE + 1

#     print(f"\rRunning batch {batch_num}/{total_batches}: {n_batch} simulations", end="", flush=True)

#     beads, states, thetas = LG_PybindWrap.simulate_multithreaded(n_batch, 32)

# print(f"\nTotal time: {time.time() - start_time:.2f} s")


In [ ]:

# time1_total, time1_mean = run_simulations(LG_PybindWrap, MAX_STEPS)


In [ ]:
# LG_Python = LangevinGillespie()
# initialize_simulation_params(LG_Python)
# time2_total, time2_mean = run_simulations(LG_Python, MAX_STEPS)

In [ ]:
# print(f"The C++ wrap was {time2_total / time1_total:.2f} times faster than Python!")